<a href="https://colab.research.google.com/github/hyunjaecho94/deepmutation/blob/master/c10_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch, torchvision, matplotlib
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn as nn
torch.manual_seed(0)

from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
np.random.seed(0)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.CIFAR10(root='CIFAR10/data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.CIFAR10(root='CIFAR10/data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


0it [00:00, ?it/s]

170500096it [00:06, 26360940.23it/s]                               


Extracting CIFAR10/data/cifar-10-python.tar.gz to CIFAR10/data/


In [0]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(2048, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [10]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/500], Loss: 1.6927
Epoch [1/5], Step [200/500], Loss: 1.3272
Epoch [1/5], Step [300/500], Loss: 1.2803
Epoch [1/5], Step [400/500], Loss: 1.2252
Epoch [1/5], Step [500/500], Loss: 1.2612
Epoch [2/5], Step [100/500], Loss: 0.7983
Epoch [2/5], Step [200/500], Loss: 1.0168
Epoch [2/5], Step [300/500], Loss: 1.0653
Epoch [2/5], Step [400/500], Loss: 1.0935
Epoch [2/5], Step [500/500], Loss: 0.8764
Epoch [3/5], Step [100/500], Loss: 1.0516
Epoch [3/5], Step [200/500], Loss: 0.9276
Epoch [3/5], Step [300/500], Loss: 0.8987
Epoch [3/5], Step [400/500], Loss: 0.9702
Epoch [3/5], Step [500/500], Loss: 0.8058
Epoch [4/5], Step [100/500], Loss: 0.9588
Epoch [4/5], Step [200/500], Loss: 0.9094
Epoch [4/5], Step [300/500], Loss: 0.6503
Epoch [4/5], Step [400/500], Loss: 0.5673
Epoch [4/5], Step [500/500], Loss: 0.8931
Epoch [5/5], Step [100/500], Loss: 0.7457
Epoch [5/5], Step [200/500], Loss: 0.7997
Epoch [5/5], Step [300/500], Loss: 0.7185
Epoch [5/5], Step [400/500], Loss:

In [0]:
correction_dict = {}
correction_dict[0] = []
correction_dict[1] = []
correction_dict[2] = []
correction_dict[3] = []
correction_dict[4] = []
correction_dict[5] = []
correction_dict[5] = []
correction_dict[6] = []
correction_dict[7] = []
correction_dict[8] = []
correction_dict[9] = []

correct_test_dataset = []

In [12]:
# Test the model
model.eval()  # eval mode
with torch.no_grad():
    correct = 0
    total = 0
    for idx, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        for j in range(batch_size):
          if predicted[j] == labels[j]:
            correction_dict[labels[j].item()].append(idx*batch_size+j)
            correct_test_dataset.append(idx*batch_size+j)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 62.72 %


In [13]:
len(correct_test_dataset)

6272

In [0]:
# Save correctly predicted test datasets
path = F"/content/gdrive/My Drive/c10_correct_test_dataset.txt" 
np.savetxt(path, correct_test_dataset, delimiter=",", fmt='%i')
path = F"/content/gdrive/My Drive/c10_correction_dict.npy" 
np.save(path, correction_dict)


In [0]:
# Save the weights of the model's layers
path = F"/content/gdrive/My Drive/c10_layer1_weights.npy" 
np.save(path, model.layer1[0].weight.data.cpu())
path = F"/content/gdrive/My Drive/c10_layer2_weights.npy" 
np.save(path, model.layer2[0].weight.data.cpu())
path = F"/content/gdrive/My Drive/c10_fc_weights.npy" 
np.save(path, model.fc.weight.data.cpu())
